In [ ]:
import boto3
import pandas as pd
from datetime import datetime, timedelta
import pytz

<b>Get detailed information about dashboard, sheets, and visuals

In [ ]:
def get_quicksight_details(dashboard_id):
    """Get detailed information about dashboard, sheets, and visuals"""
    try:
        quicksight = boto3.client('quicksight')
        account_id = boto3.client('sts').get_caller_identity()['Account']
        region = boto3.session.Session().region_name

        # Get dashboard details
        dashboard = quicksight.describe_dashboard(
            AwsAccountId=account_id,
            DashboardId=dashboard_id
        )['Dashboard']
        
        # Get dashboard definition
        definition = quicksight.describe_dashboard_definition(
            AwsAccountId=account_id,
            DashboardId=dashboard_id
        )['Definition']

        dashboard_info = {
            'DashboardId': dashboard_id,
            'DashboardName': dashboard['Name'],
            'Sheets': {}
        }

        # Extract sheet and visual information
        for sheet in definition['Sheets']:
            sheet_id = sheet['SheetId']
            
            dashboard_info['Sheets'][sheet_id] = {
                'SheetName': sheet['Name'],
                'Visuals': {}
            }

            for visual in sheet.get('Visuals', []):
                for items in visual:
                    title_root = visual.get(items,{})
                    title = title_root.get('Title',{})
                    FormatText = title.get('FormatText',{})
                    VisualTitle = FormatText.get('PlainText',"")
                    if VisualTitle=="":
                        print("Missing Title for visual id")
                    else:
                        dashboard_info['Sheets'][sheet_id]['Visuals'][title_root.get('VisualId',"")]= {'Title':VisualTitle}

        return dashboard_info

    except Exception as e:
        print(f"Error getting QuickSight details for dashboard {dashboard_id}: {str(e)}")
        return None

<b>Get list of all QuickSight dashboards with metrics

In [ ]:
def get_all_dashboards():
    """Get list of all QuickSight dashboards with metrics"""
    try:
        cloudwatch = boto3.client('cloudwatch')
        dashboards = set()
        
        paginator = cloudwatch.get_paginator('list_metrics')
        iterator = paginator.paginate(
            Namespace='AWS/QuickSight',
            MetricName='VisualLoadTime'
        )
        
        for page in iterator:
            for metric in page['Metrics']:
                for dimension in metric['Dimensions']:
                    if dimension['Name'] == 'DashboardId':
                        dashboards.add(dimension['Value'])
        
        return list(dashboards)
    except Exception as e:
        print(f"Error getting dashboards: {str(e)}")
        return []


<b> Get CloudWatch metrics for a specific visual and dashboard </b><br>
    Parameters:
    
        dasbhoard_info: which is the dictionary of all dashboard and related details.
        sheet_id
        visual_id
        start_time
        end_time
        

In [ ]:
def get_metrics_for_visual(dashboard_info, sheet_id, visual_id, start_time, end_time):
    """Get CloudWatch metrics for a specific visual and dashboard"""
    try:
        cloudwatch = boto3.client('cloudwatch')
        
        response = cloudwatch.get_metric_data(
            MetricDataQueries=[
                {
                    'Id': 'visual_avg_loadtime',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/QuickSight',
                            'MetricName': 'VisualLoadTime',
                            'Dimensions': [
                                {'Name': 'DashboardId', 'Value': dashboard_info['DashboardId']},
                                {'Name': 'SheetId', 'Value': sheet_id},
                                {'Name': 'VisualId', 'Value': visual_id}
                            ]
                        },
                        'Period': 86400,
                        'Stat': 'Average'
                    }
                },
                {
                    'Id': 'visual_max_loadtime',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/QuickSight',
                            'MetricName': 'VisualLoadTime',
                            'Dimensions': [
                                {'Name': 'DashboardId', 'Value': dashboard_info['DashboardId']},
                                {'Name': 'SheetId', 'Value': sheet_id},
                                {'Name': 'VisualId', 'Value': visual_id}
                            ]
                        },
                        'Period': 86400,
                        'Stat': 'Maximum'
                    }
                },
                {
                    'Id': 'dashboard_avg_loadtime',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/QuickSight',
                            'MetricName': 'DashboardViewLoadTime',
                            'Dimensions': [
                                {'Name': 'DashboardId', 'Value': dashboard_info['DashboardId']}
                            ]
                        },
                        'Period': 86400,
                        'Stat': 'Average'
                    }
                },
                {
                    'Id': 'dashboard_max_loadtime',
                    'MetricStat': {
                        'Metric': {
                            'Namespace': 'AWS/QuickSight',
                            'MetricName': 'DashboardViewLoadTime',
                            'Dimensions': [
                                {'Name': 'DashboardId', 'Value': dashboard_info['DashboardId']}
                            ]
                        },
                        'Period': 86400,
                        'Stat': 'Maximum'
                    }
                }
            ],
            StartTime=start_time,
            EndTime=end_time
        )

        data = []
        if 'MetricDataResults' in response:
            sheet_info = dashboard_info['Sheets'][sheet_id]
            visual_title = sheet_info['Visuals'][visual_id]['Title']

            for i, timestamp in enumerate(response['MetricDataResults'][0]['Timestamps']):
                data.append({
                    'Date': timestamp,
                    'Dashboard Name': dashboard_info['DashboardName'],
                    'Sheet Name': sheet_info['SheetName'],
                    'Visual Name': visual_title,
                    'Visual_Average_LoadTime_ms': response['MetricDataResults'][0]['Values'][i],
                    'Visual_Max_LoadTime_ms': response['MetricDataResults'][1]['Values'][i],
                    'Dashboard_Average_LoadTime_ms': response['MetricDataResults'][2]['Values'][i],
                    'Dashboard_Max_LoadTime_ms': response['MetricDataResults'][3]['Values'][i]
                })

        return data

    except Exception as e:
        print(f"Error getting metrics for visual {visual_id}: {str(e)}")
        return []


<b>     Save DataFrame to S3 in both CSV and Excel formats
    
    Parameters:
    df: pandas DataFrame to save
    bucket_name: S3 bucket name
    file_prefix: prefix for the file name (e.g., 'quicksight/metrics')

In [ ]:
def save_to_s3(df, bucket_name, file_prefix):

    try:
        s3 = boto3.client('s3')
        current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Save as CSV
        csv_buffer = df.to_csv(index=False).encode()
        csv_key = f"{file_prefix}/quicksight_metrics_{current_time}.csv"
        s3.put_object(
            Bucket=bucket_name,
            Key=csv_key,
            Body=csv_buffer
        )
        print(f"CSV file saved to s3://{bucket_name}/{csv_key}")
        
        # Save as Excel - using the correct method
        excel_path = f"/tmp/temp_{current_time}.xlsx"
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            df.to_excel(writer, index=False, sheet_name='QuickSight Metrics')
        
        excel_key = f"{file_prefix}/quicksight_metrics_{current_time}.xlsx"
        with open(excel_path, 'rb') as f:
            s3.put_object(
                Bucket=bucket_name,
                Key=excel_key,
                Body=f.read()
            )
        print(f"Excel file saved to s3://{bucket_name}/{excel_key}")
        
        # Clean up temporary file
        import os
        os.remove(excel_path)
        
        return {
            'csv_path': f"s3://{bucket_name}/{csv_key}",
            'excel_path': f"s3://{bucket_name}/{excel_key}"
        }
        
    except Exception as e:
        print(f"Error saving files to S3: {str(e)}")
        return None


<b> Analyze performance for all visuals in a dashboard/dashboards and save to S3. Average and max loadtime for dashbaord and visuals and fetched in the function

In [ ]:
def analyze_visual_performance(months=6, indashboardId=None, s3_bucket=None, s3_prefix='quicksight/metrics'):
    """Analyze performance for all visuals across all dashboards and save to S3"""
    try:
        end_time = datetime.now(pytz.UTC)
        start_time = end_time - timedelta(days=30 * months)
        
        print(f"Analyzing visual performance from {start_time.date()} to {end_time.date()}")
        
        if indashboardId == None:
            dashboards = get_all_dashboards()
        else:
            dashboards = [indashboardId]

        print(f"Analysing performance for {len(dashboards)} dashboard(s)")
        
        all_data = []
        
        for dashboard_id in dashboards:
            print(f"\nProcessing dashboard: {dashboard_id}")
            
            dashboard_info = get_quicksight_details(dashboard_id)
            if not dashboard_info:
                continue
                
            print(f"Dashboard Name: {dashboard_info['DashboardName']}")
            
            for sheet_id, sheet_info in dashboard_info['Sheets'].items():
                print(f"Processing sheet: {sheet_info['SheetName']}")
                
                for visual_id in sheet_info['Visuals'].keys():
                    data = get_metrics_for_visual(
                        dashboard_info,
                        sheet_id,
                        visual_id,
                        start_time,
                        end_time
                    )
                    all_data.extend(data)

        # Convert to DataFrame
        df = pd.DataFrame(all_data)
        
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date']).dt.date
            df = df.sort_values(['Date', 'Dashboard Name', 'Sheet Name', 'Visual Name'])
            df = df.round(2)
            
            # Save to S3 if bucket is provided
            if s3_bucket:
                save_paths = save_to_s3(df, s3_bucket, s3_prefix)
                if save_paths:
                    print("\nFiles saved successfully:")
                    print(f"CSV: {save_paths['csv_path']}")
                    print(f"Excel: {save_paths['excel_path']}")
            
        return df

    except Exception as e:
        print(f"Error in analysis: {str(e)}")
        return pd.DataFrame()

<b> The main function. You have the choice to evaluate for one visual or all visuals.

In [ ]:
def main():
    all_visuals = input("Do you want to analyze all visuals? (y/n): ").lower()
    
    if all_visuals == 'n':
        dashboardId = input("Enter dashboard ID: ")
    else:
        dashboardId = None
    
    s3_bucket = input("Enter S3 bucket name for saving results (press Enter to skip): ").strip()
    
    if s3_bucket:
        s3_prefix = input("Enter S3 prefix (default: quicksight/metrics): ").strip() or 'quicksight/metrics'
    else:
        s3_prefix = None
    
    months = int(input("Enter number of months to analyze (default: 6): ") or 6)
    
    df = analyze_visual_performance_s3save(months, dashboardId, s3_bucket, s3_prefix)
    
    if df.empty:
        print("No data found for the specified period.")
    else:
        print(f"\nAnalyzed {len(df)} records.")

if __name__ == "__main__":
    main()